# Synth. Obs.: Analytic disk

We create synthetic observations for the Magritte model of the analytic spiral that was created in the [this example](../0_creating_models/0_create_analytic_disk.ipynb).

## Setup

Import the required functionalty.

In [1]:
import magrittetorch.model.model     as magritte   # Core functionality
import magrittetorch.algorithms.solvers as solvers       # Plotting
import magrittetorch.tools.plot    as plot      # Save fits
import os
import torch

from astropy import units, constants              # Unit conversions

Define a working directory (you will have to change this). We assume here that the scripts of the [this example](../0_creating_models/0_create_analytic_disk.ipynb) have already been executed and go back to that working directory.

In [2]:
wdir = "/lhome/thomasc/Magrittetorch-examples/Analytic_disk/"

Define file names.

In [3]:
model_file = os.path.join(wdir, 'model_analytic_disk.hdf5')   # Analytic spiral Magritte model

Load the Magritte model.

In [4]:
model = magritte.Model(model_file)
model.read(legacy_mode=False) # Load Magrittetorch model

Reading model from:  /lhome/thomasc/Magrittetorch-examples/Analytic_disk/model_analytic_disk.hdf5
Reading Magrittetorch model


## Model the medium

Initialize the model.

In [5]:
model.dataCollection.infer_data() # Correctly initialize all data

In this example we will work with the LTE level populations and **do not demand** statistical equilibrium.

In [6]:
# If you have a GPU, we can use it to speed up the computations
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Getting the full model on GPU might be a bit too much for its memory, so you might want to use your CPU instead for the computations
# device = torch.device("cpu")

# Iterate level populations until statistical equilibrium
# solvers.compute_level_populations(model, device=device, 20, True)

## Make synthetic observations

Now we can make synthetic observations of the model.

In [7]:
# Get the line frequency corresponding to the CO 2-1 line
fcen = model.sources.lines.lineProducingSpecies[0].linedata.frequency.get()[0]
vpix = 1.0e+3 # velocity pixel size [m/s] 
dd   = vpix * 25 / constants.c.si.value
fmin = fcen - fcen*dd
fmax = fcen + fcen*dd

# Make sure that all input tensors lie on the same device (CPU or GPU)
device = torch.device("cpu")
freqs = torch.linspace(fmin, fmax, 51, dtype=torch.float64, device=device) # 25 frequency bins
raydir = torch.Tensor([0.0, 1.0, 0.0], device=device).type(torch.float64)
ray_nr = 3
raydir = model.geometry.rays.direction.get(device)[ray_nr]
solvers.image_model(model, raydir, freqs, torch.device("cpu"), Nxpix=256, Nypix=256)#using a resolution of 256x256 for the image. 

## Plot observations

Plot the resulting channel maps with matplotlib. Note that the resolutions do not match, as we here plot a zoomed in version.

In [8]:
plot.image_mpl(
    model,
    image_nr =  -1,
    zoom     = 1.3,
    npix_x   = 256,
    npix_y   = 256,
    x_unit   = units.au,
    v_unit   = units.km / units.s)

100%|████████████████████████████████████████████████████| 51/51 [00:26<00:00,  1.91it/s]


interactive(children=(IntSlider(value=25, description='v', max=50), Output()), _dom_classes=('widget-interact'…

<function magrittetorch.tools.plot.image_mpl.<locals>.<lambda>(v)>

(The plot is only interactive in a live notebook.)

Save the image cube in a fits file.

In [9]:
plot.save_fits(model)

Written file to: /lhome/thomasc/Magrittetorch-examples/Analytic_disk/images/image.fits


(Optional: To create your own plots) Overview of data stored in the Image object

In [10]:
xdir = model.images[-1].image_direction_x.get_astropy()#directions of the x-and y-vectors of the image
ydir = model.images[-1].image_direction_y.get_astropy()
zdir = model.images[-1].image_direction_z.get_astropy()#this is direction in which we observe the object
print("image directions: ", xdir, ydir, zdir)
nfreqs = model.images[-1].nfreqs.get() #number of frequency bins
freqs = model.images[-1].freqs.get_astropy() #frequency bins [Hz]
print("# of frequencies: ", nfreqs, " frequencies :", freqs)
ImX = model.images[-1].imX.get_astropy()#X position in image [m]
ImY = model.images[-1].imY.get_astropy()#Y position in image [m]
I = model.images[-1].I.get_astropy()#Intensity at the corresponding ImX, ImY position [W/(m^2*Hz*sr)], at a given frequency bin
# print("Intensities :", I, " ImX:", ImX, "ImY:", ImY) #prints a lot of output

image directions:  [ 0.61267884 -0.79033198  0.        ] [-0.65352411 -0.50662305 -0.5623516 ] [ 0.44444445  0.34454093 -0.82689823]
# of frequencies:  51  frequencies : [1.15261592e+11 1.15261977e+11 1.15262361e+11 1.15262746e+11
 1.15263130e+11 1.15263515e+11 1.15263899e+11 1.15264284e+11
 1.15264668e+11 1.15265053e+11 1.15265437e+11 1.15265822e+11
 1.15266206e+11 1.15266591e+11 1.15266975e+11 1.15267360e+11
 1.15267744e+11 1.15268129e+11 1.15268513e+11 1.15268898e+11
 1.15269282e+11 1.15269667e+11 1.15270051e+11 1.15270436e+11
 1.15270820e+11 1.15271205e+11 1.15271589e+11 1.15271974e+11
 1.15272358e+11 1.15272743e+11 1.15273127e+11 1.15273512e+11
 1.15273896e+11 1.15274281e+11 1.15274665e+11 1.15275050e+11
 1.15275434e+11 1.15275819e+11 1.15276203e+11 1.15276588e+11
 1.15276972e+11 1.15277357e+11 1.15277741e+11 1.15278126e+11
 1.15278510e+11 1.15278895e+11 1.15279279e+11 1.15279664e+11
 1.15280048e+11 1.15280433e+11 1.15280817e+11] Hz
